### Checklist

* Re-name header to ROI
* The script and the csv file has to be in the same folder, if not modify path
* Enter correct values for baseline and treatments
* For all cells with "plot_" manually select the ROIs 
* Cross-check logic in cells where data frames are being merged
* Change labels appropriately in summary table
* Name the export sheets appropriately


In [ ]:
import pandas as pd
import openpyxl 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
#import csv
raw = "" # enter the correct path

In [ ]:
# Defining the functions

def Baseline(start, end):
    raw_df = pd.read_excel(raw)
    baseline = raw_df.loc[(raw_df["Time (sec)"] >= start) & (raw_df["Time (sec)"] <= end)]
    baseline = baseline.set_index('Time (sec)')
    baseline = np.std(baseline)
    return baseline

### Cut-off is 6*STDEV

def Responders(start, end):   
#Selecting the duration of treatment
    duration = raw_df.loc[(raw_df["Time (sec)"] >= start) & (raw_df["Time (sec)"] <= end)]
    duration = duration.set_index("Time (sec)")
    duration_max = duration.max()
    
#Determining responders
    responders = duration_max > 6*baseline
    responders = responders.transpose().reset_index()
    responders.columns=["cells", "response"]
    responders = responders.loc[responders["response"] == True]
    return responders

In [ ]:
#Treatment 1
baseline = Baseline(0, 19) # first 20 seconds of the treatment as baseline
t1_responders = Responders(20, 100)
t1_responders

In [ ]:
plot_t1 = raw_df[["Time (sec)", \
                        "", "", ""]]

In [ ]:
#Treatment 2
baseline = Baseline(101, 119) 

t2_responders = Responders(120, 200) 
t2_responders

In [ ]:
plot_t2 = raw_df[["Time (sec)", \
                        "", "", "", "", "", "", ""]]

In [ ]:
#Treatment 3
baseline = Baseline(201, 219)

t3_responders = Responders(220, 300)
t3_responders

In [ ]:
plot_t3 = raw_df[["Time (sec)", \
                        "", "", ""]]

In [ ]:
#Treatment 4
baseline = Baseline(301, 319)

t4_responders = Responders(320, 400)
t4_responders

In [ ]:
plot_t4 = raw_df[["Time (sec)", \
                        "", "", "", ""]]

In [ ]:
#cells that respond to beta-alanine but not saline
only_ba = pd.merge(t2_responders, t1_responders, on='cells', how='outer')
only_ba = only_ba.rename(columns={"response_x": "Beta-alanine", "response_y": "Saline"}).dropna(subset=["Beta-alanine"])
only_ba

In [ ]:
#Removing cells that responded to both treatments
only_ba = only_ba.drop([4]) #index the row where both columns have a true value
only_ba

In [ ]:
plot_only_ba = raw_df[["Time (sec)", \
                        "", "", "", "", "", ""]]

In [ ]:
#cells that repsond to capsaicin but not ethanol
only_cap = pd.merge(t4_responders, t3_responders, on='cells', how='outer')
only_cap = only_cap.rename(columns={"response_x": "Capsaicin", "response_y": "Ethanol"}).dropna(subset=["Capsaicin"])
only_cap

In [ ]:
#Removing cells that responded to both treatments
only_cap = only_cap.drop([0]) #index the row where both columns have a true value
only_cap

In [ ]:
plot_only_cap = raw_df[["Time (sec)", \
                        "", "", ""]]

In [ ]:
#cells that responded to both capsaicin and any beta-alanine
ba_cap = pd.merge(only_ba[["cells","Beta-alanine"]], only_cap[["cells","Capsaicin"]], on="cells", how="inner")
ba_cap

In [ ]:
plot_ba_cap = raw_df["Time (sec)"]

In [ ]:
#Determining % of reponders

area_Nav_GCaMP = 0.00051

area_of_DRG = float(input("What is the area?: "))


no_of_neurons = area_of_DRG * area_Nav_GCaMP


per_t1_responders = round((t1_responders["cells"].count()/no_of_neurons)*100, 2)
per_t3_responders = round((t3_responders["cells"].count()/no_of_neurons)*100, 2)

per_only_ba_responders = round((only_ba["cells"].count()/no_of_neurons)*100, 2)
per_only_cap_responders = round((only_cap["cells"].count()/no_of_neurons)*100, 2)
per_ba_cap_responders = round((ba_cap["cells"].count()/no_of_neurons)*100, 2)


In [ ]:
treatment_summary = pd.DataFrame({"Saline": [per_t1_responders],
                      "Beta alanine (100mM)": [per_only_ba_responders],
                      "Ethanol": [per_t3_responders],
                      "Capsaicin (10 mM)": [per_only_cap_responders],
                      "Beta alanine and capsaicin": [per_ba_cap]})

treatment_summary


In [ ]:
# Exporting files

with pd.ExcelWriter('output.xlsx') as writer:  
    plot_t1.to_excel(writer, sheet_name='t1', index=False)
    plot_t2.to_excel(writer, sheet_name='t2', index=False)
    plot_t3.to_excel(writer, sheet_name='t3', index=False)
    plot_t4.to_excel(writer, sheet_name='t4', index=False)
    plot_only_ba.to_excel(writer, sheet_name='only_ba', index=False)
    plot_only_cap.to_excel(writer, sheet_name='only_cap', index=False)
    plot_ba_cap.to_excel(writer, sheet_name='ba_cap', index=False)    